In [1]:
import cv2 as cv
import numpy as np
import sys
# import cv2
bdcn_path = 'home/kohyoung/edge_detection/bcdn/BDCN'
sys.path.append(bdcn_path)
from utils import *
from canny import *
from thresh import *
from network import *
from dsu import *
from histogram import *

import pickle


# Load an example image
root = '/home/kohyoung/edge_detection/Data/Mobile - Black board - Shield - Narrow gaps 5'
model_root = '/home/kohyoung/edge_detection/bcdn/final-model/bdcn_pretrained_on_bsds500.pth'
array_index=3

image_dir =root+'/bmps/'
height_dir = root+'/height maps/RAW/'
img_arr=[]


raws_list = sorted(os.listdir(height_dir))
image_list = sorted(os.listdir(image_dir))
count=0
data_arr = []

#Load data and images in an array
for i in range(len(raws_list)):
    name=raws_list[i].split('.')[0]
    init_name = name[-7:]
    img_arr.append([])
    
    #Load the top mid and bottom image
    #and apply adaptive histogram equalization
    img = cv.imread(image_dir+init_name+'-Top.bmp')
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    img_arr[i].append(img_adapteq)
    
    img = cv.imread(image_dir+init_name+'-Mid.bmp')
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    img_arr[i].append(img_adapteq)
    
    img = cv.imread(image_dir+init_name+'-Bot.bmp')
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    img_arr[i].append(img_adapteq)
                      
    # prevent overlap of y-axis labels
    # plt.subplots_adjust(wspace=0.4)
    # plt.show()
   
    #Load Height Data and reshape it to the size of the image
    with open(height_dir+raws_list[i],'rb') as f:
        data = np.fromfile(f, dtype=np.float32)
        print(len(data))
        array = np.reshape(data, img.shape[:2])
        data_arr.append(array)


threshed_map_low=[]
threshed_map_high=[]

height_map = data_arr[array_index]
#Normalize Height data
height_map = np.uint8(((height_map-height_map.min())/height_map.max())*255)
ret,thr_low,thr_high = thresh_data(height_map)

#Get edges from running neural network on the data
res,combined_edge = get_edges_network(img_arr[array_index],model_root)
# morphed_edge = apply_morphing(combined_edge)

#get contours on the results fo the networks
print(combined_edge.shape)
contours_edges,contour_canny,rect_list_edges = get_bounding_box(edge_map\
    =combined_edge,color_map=(0,0,255),area_upper_lim=50000,area_lower_lim=100)
#Get bounding boxes from height data 
rect_list_height = iterative_thresh(data_arr[array_index])

#Get canny edges for finer details
edges = find_canny(img_arr)
canny_edge_arr = canny_detector(edges)

#Refine edge detection results
combined_list = refine_bb(rect_list_height)
contours_edges,contour_canny,rect_list_edges = get_bounding_box(canny_edge_arr[array_index],\
    color_map=(0,0,255),area_upper_lim=20000,area_lower_lim=100)

#Save the processed data
save_dir ='../Processed_data/'
save_file = save_dir+root.split('/')[-1]+'.txt'
value_dict = {}
value_dict['boxes']=combined_list
value_dict['image']=img_arr[array_index][1] 
if value_dict:
    with open(save_file, 'wb') as dict_items_save:
        pickle.dump(value_dict, dict_items_save)


# #Draw edges on the image data for visualization
# draw_rect(rect_list_edges,deepcopy(img_arr[array_index][1]))
# #   print(len(rect_list_edges),len(rect_list_height))
# draw_rect(combined_list,deepcopy(img_arr[array_index][1]))


ModuleNotFoundError: No module named 'torch'